In [25]:
import xarray as xr
import numpy as np
import matplotlib
# Use an interactive backend if you like, or non-interactive if running on a remote server:
import matplotlib.pyplot as plt
import datetime

# ---------------------------
# Define file paths (modify as needed)
# ---------------------------
original_path = "/gws/smf/j04/cds_c3s_lakes/LAURA/TIME_SERIES_PL_L3C/PER_LAKE_TIME_SERIES/LAKE_TS/v2.6.1-146-gfe50b81_RES20_v124_rep/LAKE000000012-ESACCI-v2.0.2_v2.1_EOCIS-L3S-LSWT-CDR-4.5-fv01.0.nc"
climatology_path = "/gws/smf/j04/cds_c3s_lakes/LAURA/TIME_SERIES_PL_L3C/PER_LAKE_CLIM_OBS/LAKE_CLIM_OBS/v2.6.1-146-gfe50b81_RES20_1995_2020/LAKE000000012_CLIM.nc"
processed_path = "/gws/nopw/j04/eocis_chuk/shaerdan/lake/lake_data/dineof_test_12_coarse/prepared.nc"

# ---------------------------
# Load the datasets
# ---------------------------
ds_orig = xr.open_dataset(original_path)
ds_proc = xr.open_dataset(processed_path)
ds_clim = xr.open_dataset(climatology_path)

# ---------------------------
# Automatically select a pixel with at least 300 non-empty (non-zero) entries in ds_proc.
# ---------------------------
non_empty_counts = (ds_proc["lake_surface_water_temperature"] != 0).sum(dim="time")
counts_np = non_empty_counts.values  # shape: (nlat, nlon)
i, j = np.where(counts_np >= 300)
if len(i) == 0:
    raise ValueError("No pixel found with at least 300 non-empty entries.")
lat_index = 20
lon_index = 35
print("Selected pixel at indices: lat_index = {}, lon_index = {}".format(lat_index, lon_index))


# ---------------------------
# Extract time series from the processed (subtracted) dataset
# ---------------------------
ts_proc = ds_proc["lake_surface_water_temperature"].isel(lat=lat_index, lon=lon_index)
print(ts_proc.values)
# ---------------------------
# Build a time axis for the processed dataset
# ---------------------------
# In the preprocessor, time was converted to days since 1981-01-01 12:00:00.
ref_date = datetime.datetime(1981, 1, 1, 12, 0, 0)
# Create a list of datetime objects from the processed time values
time_proc = [ref_date + datetime.timedelta(days=int(t)) for t in ds_proc["time"].values]
# Convert the list to a NumPy array of dtype datetime64[ns]
time_proc_dt = np.array(time_proc, dtype="datetime64[ns]")

# ---------------------------
# Extract time series from the original dataset.
# Use the converted processed time coordinate for selection.
# ---------------------------
ts_orig = ds_orig["lake_surface_water_temperature"].sel(time=time_proc_dt, method="nearest") \
            .isel(lat=lat_index, lon=lon_index)
print(ts_orig.values)
# ---------------------------
# For the climatology, we need to get the value for each processed time.
# The climatology file has 366 entries (one per day-of-year).  
# Here we compute the day-of-year for each processed time and then select the nearest climatology slice.
# ---------------------------
# Ensure the climatology variable has a coordinate "doy"
clim_da = ds_clim["lswt_mean_trimmed_345"]
if "doy" not in clim_da.coords:
    clim_da = clim_da.rename({"time": "doy"})
    clim_da = clim_da.assign_coords(doy=np.arange(1, clim_da.sizes["doy"] + 1))

# Compute day-of-year for each processed time (using the datetime objects)
doys = [t.timetuple().tm_yday for t in time_proc]

# For each time in the processed dataset, select the climatology slice using the day-of-year.
# Use nearest-neighbor selection.
ts_clim_values = []
for doy in doys:
    clim_val = clim_da.sel(doy=doy, method="nearest").isel(lat=lat_index, lon=lon_index).values
    ts_clim_values.append(clim_val)
# Wrap the climatology values in a DataArray, using the same time coordinate as the processed dataset
ts_clim = xr.DataArray(ts_clim_values, dims=["time"], coords={"time": time_proc_dt})
print(ts_clim.values)

# print(ts_orig.values)
# print(ts_clim.values)
# print(ts_proc.values)

# ----------------------------------------------------
# Filter out NaN values and prepare a common time array
# ----------------------------------------------------
# Convert the processed time list (time_proc) to a NumPy array for filtering.
time_array = np.array(time_proc)

# Create masks for non-NaN values for each time series.
mask_orig = ~np.isnan(ts_orig.values)
mask_clim = ~np.isnan(ts_clim.values)
mask_proc = ~np.isnan(ts_proc.values)

# Filter the time coordinate based on each mask.
time_orig_filtered = time_array[mask_orig]
time_clim_filtered = time_array[mask_clim]
time_proc_filtered = time_array[mask_proc]

# ----------------------------------------------------
# Plot 1: Original Data (in Kelvin)
# ----------------------------------------------------
plt.figure(figsize=(12, 6))
plt.plot(time_orig_filtered, ts_orig.values[mask_orig], 'o-', label="Original")
plt.xlabel("Time")
plt.ylabel("Temperature (K)")
plt.title("Original Data at Pixel (lat index {}, lon index {})".format(lat_index, lon_index))
plt.legend()
plt.tight_layout()
plt.savefig("original_plot.png")
plt.show()
print("Original plot saved as original_plot.png")

# ----------------------------------------------------
# Plot 2: Climatology Data (in Kelvin)
# ----------------------------------------------------
plt.figure(figsize=(12, 6))
plt.plot(time_clim_filtered, ts_clim.values[mask_clim], 'o-', label="Climatology")
plt.xlabel("Time")
plt.ylabel("Temperature (K)")
plt.title("Climatology Data at Pixel (lat index {}, lon index {})".format(lat_index, lon_index))
plt.legend()
plt.tight_layout()
plt.savefig("climatology_plot.png")
plt.show()
print("Climatology plot saved as climatology_plot.png")

# ----------------------------------------------------
# Plot 3: Processed Data (Original minus Climatology)
# (This will show differences typically below 10 K.)
# ----------------------------------------------------
plt.figure(figsize=(12, 6))
plt.plot(time_proc_filtered, ts_proc.values[mask_proc], 'o-', label="Processed (Original - Climatology)")
plt.xlabel("Time")
plt.ylabel("Difference (K)")
plt.title("Processed Data at Pixel (lat index {}, lon index {})".format(lat_index, lon_index))
plt.legend()
plt.tight_layout()
plt.savefig("processed_plot.png")
plt.show()
print("Processed plot saved as processed_plot.png")

Selected pixel at indices: lat_index = 20, lon_index = 35
[0.94000244 0.5799866         nan ...        nan        nan 1.1700134 ]
[276.44    275.91998       nan ...       nan       nan 277.57   ]
[274.81    274.69998 274.62    ... 275.82    275.77    275.69   ]
Original plot saved as original_plot.png
Climatology plot saved as climatology_plot.png
Processed plot saved as processed_plot.png


In [26]:
expected = np.where(
    (ts_orig.values == 0) | np.isnan(ts_clim.values),
    0,
    ts_orig.values - ts_clim.values
)

# Calculate the difference between the expected and the actual processed values.
diff = ts_proc.values - expected

# Use a tolerance for floating point differences.
tolerance = 1e-6
if not np.allclose(ts_proc.values, expected, atol=tolerance, equal_nan=True):
    print("Warning: Processed values do not match the expected values.")
    print("Max absolute difference:", np.nanmax(np.abs(diff)))
else:
    print("Processed values match the expected values.")

# Optionally, print out some sample comparisons for a few time steps:
for idx in range(min(5, len(ts_proc))):
    print(f"Time index {idx}: Original = {ts_orig.values[idx]}, Climatology = {ts_clim.values[idx]}, "
          f"Expected Processed = {expected[idx]}, Processed = {ts_proc.values[idx]}")

Max absolute difference: 0.85998535
Time index 0: Original = 276.44000244140625, Climatology = 274.80999755859375, Expected Processed = 1.6300048828125, Processed = 0.94000244140625
Time index 1: Original = 275.91998291015625, Climatology = 274.6999816894531, Expected Processed = 1.220001220703125, Processed = 0.579986572265625
Time index 2: Original = nan, Climatology = 274.6199951171875, Expected Processed = nan, Processed = nan
Time index 3: Original = nan, Climatology = 274.3500061035156, Expected Processed = nan, Processed = nan
Time index 4: Original = nan, Climatology = 274.0199890136719, Expected Processed = nan, Processed = nan


In [31]:
import xarray as xr
import numpy as np
import matplotlib
# Use an interactive backend if you like, or non-interactive if running on a remote server:
import matplotlib.pyplot as plt
import datetime

# ---------------------------
# Define file paths (modify as needed)
# ---------------------------
original_path = "/gws/smf/j04/cds_c3s_lakes/LAURA/TIME_SERIES_PL_L3C/PER_LAKE_TIME_SERIES/LAKE_TS/v2.6.1-146-gfe50b81_RES20_v124_rep/LAKE000000012-ESACCI-v2.0.2_v2.1_EOCIS-L3S-LSWT-CDR-4.5-fv01.0.nc"
climatology_path = "/gws/smf/j04/cds_c3s_lakes/LAURA/TIME_SERIES_PL_L3C/PER_LAKE_CLIM_OBS/LAKE_CLIM_OBS/v2.6.1-146-gfe50b81_RES20_1995_2020/LAKE000000012_CLIM.nc"
processed_path = "/gws/nopw/j04/eocis_chuk/shaerdan/lake/lake_data/dineof_test_12_coarse/prepared.nc"

# ---------------------------
# Load the datasets
# ---------------------------
ds_orig   = xr.open_dataset(original_path)
ds_clim   = xr.open_dataset(climatology_path)
ds_proc   = xr.open_dataset(processed_path)

# ---------------------------
# Select a sample pixel (change indices as desired)
# ---------------------------
lat_index = 20
lon_index = 35

# ---------------------------
# Build the time axis from the processed dataset
# ---------------------------
# In the preprocessor, time was converted to days since 1981-01-01 12:00:00.
ref_date = datetime.datetime(1981, 1, 1, 12, 0, 0)
time_proc = [ref_date + datetime.timedelta(days=int(t)) for t in ds_proc["time"].values]
# Convert to NumPy datetime64 array
time_proc_dt = np.array(time_proc, dtype="datetime64[ns]")

# ---------------------------
# Extract the original time series at the sample pixel
# ---------------------------
# Since the times match exactly, we can select using the processed time axis.
ts_orig = ds_orig["lake_surface_water_temperature"].sel(time=time_proc_dt).isel(lat=lat_index, lon=lon_index)

# ---------------------------
# Prepare the climatology
# ---------------------------
clim_da = ds_clim["lswt_mean_trimmed_345"]
# Ensure a day-of-year coordinate exists.
if "doy" not in clim_da.coords:
    clim_da = clim_da.rename({"time": "doy"})
    clim_da = clim_da.assign_coords(doy=np.arange(1, clim_da.sizes["doy"] + 1))

# Compute day-of-year for each time in time_proc.
doys = [t.timetuple().tm_yday for t in time_proc]

# For each time step, select the corresponding climatology value at the sample pixel.
ts_clim_values = []
for doy in doys:
    clim_val = clim_da.sel(doy=doy).isel(lat=lat_index, lon=lon_index).values
    ts_clim_values.append(clim_val)
ts_clim = xr.DataArray(ts_clim_values, dims=["time"], coords={"time": time_proc_dt})

# ---------------------------
# Compute the expected processed values:
# If original == 0 or climatology is NaN, expected = 0; else expected = original - climatology.
# ---------------------------
expected = np.where((ts_orig.values == 0) | np.isnan(ts_clim.values),
                    0,
                    ts_orig.values - ts_clim.values)

# ---------------------------
# Extract the processed time series at the sample pixel
# ---------------------------
ts_proc = ds_proc["lake_surface_water_temperature"].isel(lat=lat_index, lon=lon_index)
processed_values = ts_proc.values

# ---------------------------
# Compare expected and processed values
# ---------------------------
diff = processed_values - expected
tolerance = 1e-6
if np.allclose(processed_values, expected, atol=tolerance, equal_nan=True):
    print("Processed values match the expected values.")
else:
    print("Mismatch found: maximum absolute difference is", np.nanmax(np.abs(diff)))
    # Print a few sample comparisons
    for idx in range(min(5, len(ts_proc))):
        print(f"Time index {idx}: Original = {ts_orig.values[idx]}, "
              f"Climatology = {ts_clim.values[idx]}, "
              f"Expected = {expected[idx]}, "
              f"Processed = {processed_values[idx]}")

FileNotFoundError: [Errno 2] No such file or directory: '/gws/nopw/j04/eocis_chuk/shaerdan/lake/lake_data/dineof_test_12_coarse/prepared.nc'